In [ ]:
import pandas as pd
#Script merge de deux fichiers CSV
# Charger les deux fichiers
df_gpwfirst = pd.read_csv('WHO_SDG_GPW_standardisé_avec_code_pays.csv')
df_sdg3first = pd.read_csv('WHO_SDG3_standardisé_avec_code_pays.csv')
#On sélectionne uniquement les colonnes nécessaires pour la fusion
df_sdg3= df_sdg3first[["LOCATION", "Pays_code_iso3", "Espérance de vie totale", "Espérance de vie en bonne santé totale"]]
df_gpw= df_gpwfirst[["LOCATION", "Pays_code_iso3","Dépenses publiques santé"]]
# 1. Identifier les colonnes communes (Clés de fusion)
# On suppose ici que les colonnes identifiantes ont le même nom (ex: 'ParentLocation', 'Location', 'Period')
# Ajustez cette liste selon vos vrais noms de colonnes
keys = ['LOCATION','Pays_code_iso3'] 

# Vérification rapide si les clés existent dans les deux
missing_keys = [k for k in keys if k not in df_gpw.columns or k not in df_sdg3.columns]
if missing_keys:
    print(f"Attention : Les colonnes clés {missing_keys} ne sont pas présentes dans les deux fichiers.")
else:
    # 2. Fusionner intelligemment (Merge)
    # how='outer' garde toutes les données (même si un pays manque dans l'un des deux fichiers)
    # how='inner' ne garde que les lignes présentes dans les DEUX fichiers
    df_fusionne = pd.merge(df_gpw, df_sdg3, on=keys, how='outer', suffixes=('_gpw', '_sdg3'))

    print('=== Table fusionnée (Merge sécurisé) ===')
    print(f'Shape: {df_fusionne.shape}')
    
    # Vérifier s'il y a eu duplication de lignes indésirable
    if df_fusionne.shape[0] > max(df_gpw.shape[0], df_sdg3.shape[0]):
        print("Note: Le nombre de lignes a augmenté. Vérifiez les doublons dans les clés de jointure.")

    print(df_fusionne.head())

    # Sauvegarder
    df_fusionne.to_csv('WHO_SDG_GPW_SDG3_fusionne_secure.csv', index=False)
    print('\n✓ Table fusionnée sauvegardée.')

=== Table fusionnée (Merge sécurisé) ===
Shape: (197, 5)
      LOCATION Pays_code_iso3  Dépenses publiques santé  \
0  Afghanistan            AFG                      1.13   
1      Albania            ALB                      9.19   
2      Algeria            DZA                      5.36   
3      Andorra            AND                     15.87   
4       Angola            AGO                      6.70   

   Espérance de vie totale  Espérance de vie en bonne santé totale  
0                     59.1                                    50.4  
1                     76.4                                    66.7  
2                     76.0                                    65.5  
3                      NaN                                     NaN  
4                     62.1                                    53.8  

✓ Table fusionnée sauvegardée.
